In [56]:
from funcoesTermosol import importa, plota, geraSaida, gauss_seidel
import numpy as np

[nn,N,nm,Inc,nc,F,nr,R, L, angle] = importa('entrada.xlsx')
print(R)

geraSaida('teste',F,F,F,F,F)

[[1. 1.]
 [2. 1.]
 [2. 2.]]


In [69]:
# Define a matriz de rigidez local para cada barra
K = []
nos = np.zeros((nm,2))

for i in range(nm):
    E = Inc[i,2]
    A = Inc[i,3]
    Li = L[i][0]
    theta = angle[i,0]

    nos[i,0] = Inc[i,0]
    nos[i,1] = Inc[i,1]

    K1 = np.zeros((4,4), dtype=float)

    K1[0,0] = (E*A/Li)*np.cos(theta)**2
    K1[0,1] = (E*A/Li)*np.cos(theta)*np.sin(theta)
    K1[0,2] = -(E*A/Li)*np.cos(theta)**2
    K1[0,3] = -(E*A/Li)*np.cos(theta)*np.sin(theta)

    K1[1,0] = (E*A/Li)*np.cos(theta)*np.sin(theta)
    K1[1,1] = (E*A/Li)*np.sin(theta)**2
    K1[1,2] = -(E*A/Li)*np.cos(theta)*np.sin(theta)
    K1[1,3] = -(E*A/Li)*np.sin(theta)**2

    K1[2,0] = -(E*A/Li)*np.cos(theta)**2
    K1[2,1] = -(E*A/Li)*np.cos(theta)*np.sin(theta)
    K1[2,2] = (E*A/Li)*np.cos(theta)**2
    K1[2,3] = (E*A/Li)*np.cos(theta)*np.sin(theta)

    K1[3,0] = -(E*A/Li)*np.cos(theta)*np.sin(theta)
    K1[3,1] = -(E*A/Li)*np.sin(theta)**2
    K1[3,2] = (E*A/Li)*np.cos(theta)*np.sin(theta)
    K1[3,3] = (E*A/Li)*np.sin(theta)**2
    K.append(K1)

# if a number in K is between -1e-8 and 1e-8, it is considered 0
for i in range(nm):
    for j in range(4):
        for k in range(4):
            if K[i][j,k] < 1e-8 and K[i][j,k] > -1e-8:
                K[i][j,k] = 0


# Define a matriz global de rigidez
KG = np.zeros((nn*2,nn*2))

for count, matrizLocal in enumerate(K):
    n = nos[count]
    m1 = int(n[0])
    m2 = int(n[1])
    KG[np.ix_([(m1 - 1)*2, (m1 - 1)*2+1, (m2 - 1)*2, (m2 - 1)*2+1], [(m1 - 1)*2, (m1 - 1)*2+1, (m2 - 1)*2, (m2 - 1)*2+1])] += matrizLocal[0:4,0:4]

for r in R:
    KG[int(r[0]-1)*2  + (int(r[1]) - 1), :] = 0
    KG[:, int(r[0]-1)*2  + (int(r[1]) - 1)] = 0

# KGfinal = KG[np.ix_([1,4,5],[1,4,5])]
notzero = np.arange(0,nn*2)
for i in range(len(R)):
    if (R[i,0]-1)*2 + (R[i,1] - 1) in notzero:
        notzero = np.delete(notzero, np.where(notzero == (R[i,0]-1)*2 + (R[i,1] - 1)))

print(notzero)
KGfinal = KG[np.ix_(notzero,notzero)]

fglobal = F.copy()
fglobal  = fglobal[np.ix_(notzero)]
print(fglobal)
print(KGfinal)

[1 4 5]
[[   0.]
 [ 150.]
 [-100.]]
[[ 1.5876e+08 -4.0320e+07 -5.3760e+07]
 [-4.0320e+07  1.7024e+08  4.0320e+07]
 [-5.3760e+07  4.0320e+07  5.3760e+07]]


In [118]:
# K x u = F

u = np.zeros((len(notzero),1))

x = gauss_seidel(KGfinal, fglobal, u, 1e-10, 1000)

utotal = np.zeros((nn*2,1))
for i in range(len(notzero)):
    utotal[notzero[i]] = x[i]

print(utotal)

[[ 0.00000000e+00]
 [-9.50945067e-07]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 1.60575208e-06]
 [-4.01537818e-06]]


In [117]:
# Deformação específica
defEsp = np.zeros((nm,1))
for i in range(nm):
    E = Inc[i,2]
    A = Inc[i,3]
    Li = L[i][0]
    theta = angle[i,0]

    defEsp[i] = (1/Li)*np.matmul(np.array([-np.cos(theta), -np.sin(theta), np.cos(theta), np.sin(theta)]), 
                                 np.array([utotal[int(nos[i,0])*2 - 2], utotal[int(nos[i,0])*2 - 1], utotal[int(nos[i,1])*2 - 2], utotal[int(nos[i,1])*2 - 1]]))

defEsp

array([[ 2.37736267e-06],
       [ 5.35250694e-06],
       [-2.97619048e-06]])